In [1]:
import pandas as pd
import numpy as np
import re
import ast
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from textblob import TextBlob

In [3]:
data = pd.read_csv('C:/Users/Camila Sajnin/Documents/teste/books_data.csv')
rating = pd.read_csv('C:/Users/Camila Sajnin/Documents/teste/Books_rating.csv')

MemoryError: 

In [ ]:
#Ver quais são as colunas das tabelas:
print("As colunas da tabela data são:")
for i in data.columns:
    print(i)
print("")
print("As colunas da tabela rating são:")
for j in rating.columns:
    print(j)  

In [ ]:
#Ver a Tabela data
display(data)

In [ ]:
#Ver a tabela rating
display(rating)

In [ ]:
# Verificar valores nulos nas colunas 
print("Os nulos da tabela data são:")
print(data.isnull().sum())  

print("")
print("Os nulos da tabela rating são:")
print(rating.isnull().sum())  

In [ ]:
#Tratamento dos nulos
data['authors'] = data['authors'].fillna('Autor não disponível')
data['image'] = data['image'].fillna('Imagem não disponível')
data['publisher'] = data['publisher'].fillna('Desconhecido')
data['categories'] = data['categories'].fillna('Não informado')
data['description'] = data['description'].fillna('Descrição não disponível')
rating['Price'] = rating['Price'].fillna(0)
rating['User_id'] = rating['User_id'].fillna('Desconhecido')
rating['profileName'] = rating['profileName'].fillna('Desconhecido')

# Remover as linhas onde os nulos são críticos
data.dropna(subset=['Title'], axis=0, inplace=True)
rating.dropna(subset=['Title'], axis=0, inplace=True)

# Remover a linha de ratings count pois contém muitos nulos e conseguimos essa informação juntando as duas tabelas
#Remover outras colunas desnecessárias
data.drop(columns=['ratingsCount','image','previewLink','publisher','infoLink'])


In [ ]:
#Definição de formato

data['Title'] = data['Title'].astype(str)
data['description'] = data['description'].astype(str)
data['authors'] = data['authors'].astype(str)
data['publishedDate'] = pd.to_datetime(data['publishedDate'], format='mixed', errors='coerce')
data['categories'] = data['categories'].astype(str)
rating['Id'] = rating['Id'].astype(str)
rating['Title'] = rating['Title'].astype(str)
rating['Price'] = rating['Price'].astype(float)
rating['User_id'] = rating['User_id'].astype(str)
rating['profileName'] = rating['profileName'].astype(str)
rating['score'] = rating['score'].astype(float)
rating['time'] = pd.to_datetime(rating['time'], unit='s')
rating['summary'] = rating['summary'].astype(str)
rating['text'] = rating['text'].astype(str)

In [ ]:
#Converter coluna em uma lista para conseguir separar os autores
data['authors'] = data['authors'].apply(lambda x: ast.literal_eval(x) if x.startswith('[') else [x])

# Explodir a coluna 'authors' para que cada autor fique em uma linha separada
data = data.explode('authors')

In [ ]:
#Quantas categorias diferentes existem
print(data['categories'].nunique())  

In [ ]:
# Função para limpar os textos 
def preprocess_text(text):
    text = text.lower().strip()                      
    text = re.sub(r'[^a-záéíóúãõç ]', '', text)       
    return text

cat_preprocessadas= data['categories'].apply(preprocess_text)

# Vetorização com TF-IDF a partir das categorias limpas
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cat_preprocessadas)

# Clusterização com KMeans
num_clusters = 11
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(X)

# Nomeação automática dos clusters usando TF-IDF
cluster_names = {}
for cluster_id in range(num_clusters):
    # Seleciona as categorias limpas correspondentes ao cluster atual
    mask = clusters == cluster_id
    categorias_no_cluster = cat_preprocessadas[mask].tolist()
    
    # Junta os textos para formar um único documento
    texto_concatenado = ' '.join(categorias_no_cluster)
    
    # Vetorização desse texto para identificar a palavra com maior peso
    tfidf = TfidfVectorizer()
    X_cluster = tfidf.fit_transform([texto_concatenado])
    termos = np.array(tfidf.get_feature_names_out())
    pesos = X_cluster.toarray().flatten()
    
    # Escolhe a palavra mais importante para nomear o cluster
    palavra_mais_importante = termos[np.argmax(pesos)]
    cluster_names[cluster_id] = palavra_mais_importante.capitalize()

# Renomear o cluster que ficou com "Informado" para "Não informado"
for cluster_id, nome in cluster_names.items():
    if nome.lower() == "informado":
        cluster_names[cluster_id] = "Não informado"

# Criar a coluna final 'categorias_agrupadas' no DataFrame utilizando o mapeamento
data['categorias_agrupadas'] = pd.Series(clusters, index=data.index).map(cluster_names)

# Exibir os grupos de categorias com seus nomes
for nome, grupo in data.groupby('categorias_agrupadas')['categories']:
    print(f"\n {nome}: {', '.join(grupo.tolist())}")


In [ ]:
# Gerar tabela com categorias de livros com a maior média de score

#Juntar as informações utilizando Titulo como chave
score_por_categoria = pd.merge(
    rating[['Title', 'score', 'summary']], 
    data[['Title', 'categorias_agrupadas']], 
    on='Title', 
    how='left'
)

# Calcular a média dos scores, o número de avaliações e concatenar os resumos
tb_media_por_categoria = score_por_categoria.groupby('categorias_agrupadas').agg(
    media_score=('score', 'mean'),
    num_avaliacoes=('score', 'count'),
    resumo_categoria=('summary', lambda textos: " ".join(textos))
).reset_index()

# Adicionar palavras que não ajudam na analise à lista de stop words
palavras_excluidas = {
    "best", "book", "excellent", "classic", "good", "great", 
    "love", "interesting", "amazing", "awesome", "favorite", "favourite", 
    "must", "read", "novel", "story", "inspiring", "popular"
}
stop_words_custom = ENGLISH_STOP_WORDS.union(palavras_excluidas)

# Função para extrair as 10 palavras mais importantes usando TF-IDF
def extract_keywords(text, max_features=10):
    if not text.strip():
        return ""
    vectorizer = TfidfVectorizer(stop_words=stop_words_custom, max_features=max_features)
    X = vectorizer.fit_transform([text])
    keywords = vectorizer.get_feature_names_out()
    return ", ".join(keywords)

# Extrair as palavras-chave do resumo agregado para cada categoria
tb_media_por_categoria['palavras_chaves'] = tb_media_por_categoria['resumo_categoria'].apply(lambda t: extract_keywords(t, max_features=10))
tb_media_por_categoria.drop(columns=['resumo_categoria'], inplace=True)

display(tb_media_por_categoria)


In [ ]:
# Existem autores que escrevem em mais de uma categoria?

data_filtrado = data[data['categorias_agrupadas'] != 'Não informado']

# Agrupar por autores e agregar as categorias
resultado = data_filtrado.groupby('authors').agg(
    categorias_distintas=('categorias_agrupadas', lambda x: ', '.join(set(x))),
    mais_de_uma_categoria=('categorias_agrupadas', lambda x: len(set(x)) > 1)
).reset_index()

display(resultado[resultado['mais_de_uma_categoria']])


In [ ]:
# Gerar tabela com autores de livros e suas categorias com a maior média de score

#Juntar as informações utilizando Titulo como chave
score_por_autor = pd.merge(rating[['Title', 'score']], data[['Title', 'authors','categorias_agrupadas']], on='Title', how='left')

# Calcular a média do score por autor e quantidade de avaliações
tb_media_por_autor = score_por_autor.groupby('authors').agg(
    media_score=('score', 'mean'),
    num_avaliacoes=('score', 'count')
).reset_index()

tb_media_por_autor = tb_media_por_autor.sort_values(by='num_avaliacoes', ascending=False)

display(tb_media_por_autor)

In [ ]:
#Gerar a média do score por livro
media_por_livro = rating.groupby('Title').agg(
    media_score = ('score','mean')
).reset_index()

In [ ]:
#Sentimento com base no summary e nota 

# Função para classificar o sentimento com base na polaridade
def categorize_sentiment(polarity):
    if polarity > 0.6:
        return "Muito Positivo"
    elif polarity > 0.2:
        return "Positivo"
    elif polarity > -0.2:
        return "Neutro"
    elif polarity > -0.6:
        return "Negativo"
    else:
        return "Muito Negativo"

# Função para calcular a polaridade de cada resumo
def analyze_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

# Calcular a polaridade para cada resumo e classificá-lo
rating['polarity'] = rating['summary'].apply(analyze_sentiment)
rating['sentiment'] = rating['polarity'].apply(categorize_sentiment)

# Combinar a polaridade com o score para verificar se faz sentido

# Função para combinar o sentimento e o score
def sentiment_with_score(sentiment, score):
    # Se o score for maior que 3 (positivo), e o sentimento for muito positivo ou positivo, é uma confirmação.
    # Se o score for baixo, o sentimento negativo é validado
    if score >= 4:
        if sentiment in ["Muito Positivo", "Positivo"]:
            return "Confirmado Positivo"
        else:
            return "Discrepância Positiva"
    elif score >= 2:
        if sentiment == "Neutro":
            return "Confirmado Neutro"
        else:
            return "Discrepância Neutra"
    else:
        if sentiment in ["Negativo", "Muito Negativo"]:
            return "Confirmado Negativo"
        else:
            return "Discrepância Negativa"

# Aplicar a função de combinação
rating['sentiment_with_score'] = rating.apply(lambda row: sentiment_with_score(row['sentiment'], row['score']), axis=1)

display(rating[['summary', 'score', 'sentiment', 'sentiment_with_score']])

In [ ]:
# Criar coluna auxiliar 'Sentimento_final'
def categorize_overall(sentiment_with_score):
    if sentiment_with_score.startswith("Confirmado"):
        return sentiment_with_score.split()[1].lower()
    else:
        return "Inconsistente"

rating['sentimento_final'] = rating['sentiment_with_score'].apply(categorize_overall)
rating.drop(columns=['sentiment','sentiment_with_score'])

#Apagar linhas com sentimentos inconsistentes
rating = rating[rating['sentimento_final'] != 'Inconsistente']


In [ ]:
# Tabela geral dos sentimentos

# Contar a frequência de cada sentimento 
tb_sentimento = rating['sentimento_final'].value_counts().reset_index()
tb_sentimento.columns = ['sentimento', 'count']

display(tb_sentimento)

In [ ]:
# Gerar tabela com os livros com mais avaliações

# Função para obter o sentimento que mais aparece
def get_mode(series):
    modes = series.mode()
    return modes.iloc[0] if not modes.empty else None

# Agrupar pelo Titulo: contar as avaliações e determinar o sentimento predominante
tb_sentimento_livro = rating.groupby('Title').agg(
    num_avaliacoes=('Title', 'count'),
    Sentimento=('sentimento_final', lambda x: get_mode(x.str.replace("Confirmado ", "")))
).reset_index()


tb_sentimento_livro = tb_sentimento_livro.sort_values(by='num_avaliacoes', ascending=False)

display(tb_sentimento_livro)

In [ ]:
#Ver datas de avaliações
display(rating.sort_values(by='time',ascending=False))

In [ ]:
# Selecionar somente as colunas necessárias de rating e media_por_livro
tb_avaliacoes = pd.merge(
    rating[['Title', 'score', 'User_id', 'profileName', 'time']],
    media_por_livro[['Title', 'media_score']],
    on='Title',
    how='left'
)

# Calcular o desvio relativo para cada avaliação: (|score do usuário - média do livro| / média do livro)
tb_avaliacoes['desvio_relativo'] = (tb_avaliacoes['score'] - tb_avaliacoes['media_score']).abs() / tb_avaliacoes['media_score']

# Agrupar os dados por usuário e agregar as informações desejadas:
assertividade_usuarios = tb_avaliacoes.groupby('User_id').agg(
    profileName=('profileName', 'first'),
    ultima_avaliacao=('time', 'max'),
    num_avaliacoes=('User_id', 'count'),
    desvio_relativo_medio=('desvio_relativo', 'mean')
).reset_index()

# Filtrar os leitores assertivos: desvio médio <= 0.15 (15% de desvio), número de avaliações >= 100 e ultima avaliacão a partir de 2012(visto que a data mais recente é 2013)
tb_leitores_assertivos = assertividade_usuarios[
    (assertividade_usuarios['desvio_relativo_medio'] <= 0.15) &
    (assertividade_usuarios['num_avaliacoes'] >= 100) &
    (assertividade_usuarios['ultima_avaliacao'] >= '2012-01-01')
]

# Ordenar os leitores pelos que mais avaliaram (em ordem decrescente)
tb_leitores_assertivos = tb_leitores_assertivos.sort_values(by='num_avaliacoes', ascending=False)

display(tb_leitores_assertivos)


In [ ]:
#Salvar tabelas para utilizar no powerbi
tabelas = {
    'tb_media_por_categoria': tb_media_por_categoria,
    'tb_media_por_autor': tb_media_por_autor,
    'tb_sentimento':tb_sentimento,
    'tb_sentimento_livro': tb_sentimento_livro,
    'tb_leitores_assertivos': tb_leitores_assertivos,
}

for nome, df in tabelas.items():
    df.to_csv(f'{nome}.csv', index=False)